In [4]:
# enumerate dates from 2008

import datetime

def yield_datetime(start: datetime.datetime, end: datetime.datetime = None):
    # endless loop if end is not specified
    if end is None:    
        while True:
            yield start
            start += datetime.timedelta(days=1)
    else:
        while start < end:
            yield start
            start += datetime.timedelta(days=1)

In [ ]:
!pip install warcio
import warcio
?warcio.WARCWriter
writer = warcio.WARCWriter(None)
?writer.create_warc_record

Signature:
writer.create_warc_record(
    uri,
    record_type,
    payload=None,
    length=None,
    warc_content_type='',
    warc_headers_dict=None,
    warc_headers=None,
    http_headers=None,
)
Docstring: <no docstring>
File:      /workspaces/handy-snippets-in-jupyter-notebook/.venv/lib/python3.12/site-packages/warcio/recordbuilder.py
Type:      method

In [ ]:
!pip install requests
import requests

# read all dates like https://unwire.hk/2025/08/21/

# save them as common crawl format (see https://commoncrawl.org/the-data/)

!pip install tqdm
from tqdm.auto import tqdm
for date in tqdm(list(yield_datetime(datetime.datetime(2008, 1, 1), end=datetime.datetime.today()))):
    url = f"https://unwire.hk/{date:%Y/%m/%d}/"
    r = requests.get(url)
    if r.status_code == 200:
        # AttributeError: module 'warcio' has no attribute 'WarcFile'
        with open("unwire.hk.warc", "ab") as f:
            writer = warcio.WARCWriter(f)
            writer.create_warc_record(uri=url)

  0%|          | 0/6443 [00:00<?, ?it/s]


TypeError: 'WARCWriter' object does not support the context manager protocol

In [20]:
!pip install beautifulsoup4
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.content, "html.parser")

In [24]:
!pip install pandas
import pandas as pd
alist = pd.Series(soup.find_all("a"))
alist

0              [[], []]
1             [[Learn]]
2       [GenAI 與雲端科技專區]
3       [ERP 與商業 AI 專區]
4     [Galaxy Z Fold 7]
            ...        
80             [網站活動報告]
81               [重點推介]
82          [unwire TV]
83          [unwire TV]
84          [Cookie 政策]
Length: 85, dtype: object

In [25]:
alist.apply(lambda x: x.get("href"))

0                                    https://unwire.hk/
1                                https://list.unwire.hk
2                    https://unwire.hk/tag/genai-cloud/
3                https://unwire.hk/tag/erp-business-ai/
4                https://unwire.hk/tag/galaxy-z-fold-7/
                            ...                        
80    https://unwire.hk/category/site_news/event-rep...
81                 https://unwire.hk/category/headline/
82           https://unwire.hk/category/unwire_podcast/
83                         https://unwire.hk/unwire-tv/
84                                                    #
Length: 85, dtype: object

In [28]:
import bs4
def get_css_selector_for_element(element: bs4.element.Tag):
    # get ancestors
    ancestors = []
    while element.parent:
        ancestors.append(element.name)
        element = element.parent
    ancestors.reverse()
    # get css selector
    css_selector = ""
    for ancestor in ancestors:
        css_selector += f"{ancestor} > "
    css_selector += element.name
    return css_selector

In [30]:
alist.apply(get_css_selector_for_element).value_counts()

html > body > header > div > div > ul > li > a > [document]                                                                   43
html > body > header > div > div > div > div > div > div > div > ul > li > li > li > div > ul > li > span > a > [document]    12
html > body > header > div > div > div > div > div > div > div > ul > li > li > div > ul > li > span > a > [document]          8
html > body > header > div > div > div > div > div > div > div > ul > li > div > ul > li > span > a > [document]               7
html > body > header > div > div > div > div > div > div > div > ul > li > li > li > span > a > [document]                     5
html > body > header > div > div > div > div > div > div > div > ul > li > span > a > [document]                               4
html > body > header > div > div > div > div > div > div > div > ul > li > li > span > a > [document]                          2
html > body > header > div > ul > li > a > [document]                                            